In [ ]:
using ModelingToolkit, ThermalSystem_Models, Logging, Revise, CoolProp, Printf, OrdinaryDiffEq
Revise.retry()
Logging.disable_logging(Logging.Warn)

TSM = ThermalSystem_Models
TSMD = TSM.Dynamics
TSMT = TSM.DeadTime
MTK = ModelingToolkit
Steam = TSMD.Steam
Gas = TSMD.Gas
Liq = TSMD.Liq

#### Testing alias elimination

CONCLUSION: Did not work - best route forward is to just remap

In [ ]:
using Unitful, Symbolics

# actual property functions
specific_prop1_func(x,y)  = PropsSI("C","P",x,"T",y*u"K","Helium").val,
specific_prop2_func(x,y)  = PropsSI("ISENTRopic_expansion_coefficient","P",x*u"bar","T",y*u"K","Helium").val

#registered as symbolics
Symbolics.@register_symbolic specific_prop1_func(x,y)
# Symbolics.@register_symbolic specific_prop2_func(x,y)

#general funciton
Symbolics.@register_symbolic general_prop1_func(x,y)
Symbolics.@register_symbolic general_prop2_func(x,y)


subdict = Dict(general_prop1_func => specific_prop1_func,
                general_prop2_func => specific_prop2_func)

@variables t
Tfunc(t) = 300.0
Pfunc(t) = 50.0
@register_symbolic Tfunc(t) 
@register_symbolic Pfunc(t)
@variables T(t) prop1(t) P(t)

equation = [T ~ Tfunc(t),
            P ~ Pfunc(t),
            prop1 ~ general_prop1_func(T,P)]

push!(equation, general_prop1_func(T,P) ~ specific_prop1_func(T,P))

@named tstsys = ODESystem(equation,t,[T,P,prop1],[])
# alias_elimination(tstsys)
slsys = alias_elimination(tstsys)
structural_simplify(slsys)

## Working on map

In [ ]:
plant_sys,cntrl_vars = TSMD.para_full_system_brayton();
TSMD.system_details(plant_sys)
@unpack powercycle,intloop = plant_sys
subsys = deepcopy(plant_sys)
# for pd in [Gas.propDict,Steam.hydro_prop_dict,Liq.propDict]
#     subsys = substitute(subsys, pd)
# end
TSMD.system_details(subsys)


In [ ]:
eqs = equations(subsys);
cntrl_vars
d = Dict()
for i = 1:5
    d[cntrl_vars[i]] = 100
end
for i = 1:3
    d[cntrl_vars[5+i]] = i*100e6
end
new_eqs = substitute(eqs,d);
@named new_sys = ODESystem(new_eqs,t);
TSMD.system_details(new_sys)

MTK.linearize(subsys,union(cntrl_vars,parameters(subsys)))

In [ ]:
AbstractTrees.children(x::ODESystem) = TSMD.systems(x)
AbstractTrees.nodevalue(x::ODESystem) = x.name
AbstractTrees.printnode(x::ODESystem) = print(x.name)
cvals(x) = [nodevalue(xx) for xx in children(x)]

In [ ]:
# @named testpart = TSMD.Gas.ActiveThermoTurbine()
print_tree(sys)

In [ ]:
lev1 = children(sys)
inst = lev1[1]
cvals(inst)
nextsibling(inst)

In [ ]:
lev2 = children(inst)
inst = lev2[1]
cvals(inst)
print_tree(sys)
# lev3 = children(inst)
# inst = lev3[1]
# cvals(inst)



In [ ]:
@named testpart = TSMD.Gas.ActiveThermoTurbine()
eqs = equations(testpart)
cpf(t) = 5192
kkf(t) = 1.6667

to_eliminate = collect(keys(Gas.propDict))

typeof(to_eliminate[1])
eqref = eqs[9]
tomathc = to_eliminate[2]

sides(x) = [x.lhs, x.rhs] 
lhs,rhs=sides(eqref)

### This is helpful

In [ ]:
v2n = powercycle.var_to_name
symnam = collect(keys(powercycle.var_to_name))

strnam = [String(s) for s in symnam]

tofind = :ṁ
tfStr = String(tofind)
flowvar = collect(tfStr)[1]
flowkeys = Symbol[]
flowvals = SymbolicUtils.BasicSymbolic{Real}[]

#   finds all system variables with ṁ
for (i,str) in enumerate(strnam)
    chr = collect(str)[end]
    if chr == flowvar
        push!(flowkeys,symnam[i])
        push!(flowvals,v2n[symnam[i]])
    end
end

In [ ]:
v2n,flow_symbol,flow_variable = TSMD.mass_flow_vars(powercycle);
eqs = TSMD.connection_equations(powercycle);
SplitStr = "₊"
eq = eqs[1]
compnames = TSMD.component_names(eqs; unique_only = true)
cdict = Dict(cc => ii for (ii,cc) in enumerate(compnames))
g = DiGraph()
add_vertices!(g,length(compnames))
cdict

In [ ]:
for eq in eqs
    vars = MTK.get_variables(eq)
    symvar = TSMD.variable2symbol(vars); # variables in the form :variablenmame
    bol = TSMD.check_for_ṁ(symvar)
    if bol == true
        # @show symvar
        cname = [TSMD.split_variable_sym(s) for s in symvar]
        portid = [String(c[end-1]) for c in cname]
        portNames = [String(c[1]) for c in cname]
        
        nIdx = findfirst(x -> x=="n",portid)
        pIdx = findfirst(x -> x=="p",portid)
        # @show Symbol(portNames[pIdx])
        if !isnothing(nIdx) && !isnothing(pIdx)
            add_edge!(g,cdict[Symbol(portNames[nIdx])], cdict[Symbol(portNames[pIdx])])
        end
    end
end


In [ ]:
using PlotlyJS, LightGraphs
import GraphPlot  # for spring_layout
pos_x, pos_y = GraphPlot.spring_layout(g)

In [ ]:
sv = symvar[end]

# sep = "₊"
# name_end = findfirst(x -> x==sep,collect(str))
# collect(str)[end-2] == 'n'
# occursin("n",str)
# String(ex.args)
# name_end

In [ ]:

# @show typeof(eqvars[1])
# @show typeof(flow_variable[1])
# # v2n[eqvars[1]]
# display(eqvars)
# display(flow_variable)
# v2n[eqvars[1]]
# # Symbol(String(eqvars[1]))
# findfirst(x -> x == eqvars[1], flow_variable)

va

In [ ]:


# vars = powercycle.var_to_name[flowidx]
# v2n = powercycle.var_to_name
# flowidx[1]
# typeof(v2n[flowidx[2]])
eqAll = equation_dependencies(powercycle)
eqFlow = equation_dependencies(powercycle, variables = flowvals);

flow_var_idx = findall(x -> !isempty(x),eqFlow);
mod_eq_flow = eqFlow[flow_var_idx];

eqdict = Dict(s => i for (i,s) in enumerate(flowvals))
as = asgraph(mod_eq_flow,eqdict)
# flowvals[19]
# asg = asdigraph(as, powercycle)


In [ ]:
using PlotlyJS, LightGraphs
import GraphPlot  # for spring_layout
pos_x, pos_y = GraphPlot.spring_layout(as)


In [ ]:
newsys = deepcopy(sys);
for pd in [Gas.propDict,Steam.hydro_prop_dict,Liq.propDict]
    newsys = substitute(newsys, pd);
end

In [ ]:
eq = equations(sys)
for pd in [Gas.propDict,Steam.hydro_prop_dict,Liq.propDict]
    eq = substitute(eq, pd);
end
@named updatedsys = ODESystem(eq, MTK.get_iv(sys), states(sys), parameters(sys));

In [ ]:
Nsys    = deepcopy(sys);
systems = TSMD.systems(Nsys)

In [ ]:
SimplePlant,Plant,copyplant,sol = TSMD.test_full_system_brayton();

In [ ]:
using Symbolics, SymbolicUtils
Eq = equations(sys)

@syms t ṁ(t) FINDME(t)
d = Dict(ṁ => FINDME)
subEq = substitute(Eq,d)

sts = states(sys)
@show typeof(sts[1])
@show istree(sts[1])

SymbolicUtils.substitute(sts[1],d)
# Symbolics.get_variables!(sts[1])
propertynames(sts[1])
# Meta.@dump(sts)
# Meta.parse(eval(nm))
exp = Meta.quot(nm)
Meta.show_sexpr(exp)
dump(exp.args)
# ex1 = Meta.parse(Symbol(nm))
# dump(sts[1])


In [ ]:
using Symbolics, SymbolicUtils

@variables